In [126]:
import itertools
from math import sqrt
import numpy as np
from numpy import arange, pi, sin, cos, arccos

GOLDEN_RATIO = (1 + 5**0.5)/2

def fibonacci_sequence_sphere(center, radius, n_points):
    i = arange(0, n_points)
    theta = 2 *pi * i / GOLDEN_RATIO
    phi = arccos(1 - 2*(i+0.5)/n_points)
    x = radius * sin(phi) * cos(theta) + center
    y = radius * sin(phi) * sin(theta) + center
    z = radius * cos(phi) + center

    lms = np.vstack((x, y, z)).T

    # rounding of matrix values to account for pixel rounding errors
    lms[lms < center - radius + 0.5] = center - radius + 0.5
    lms[lms > center + radius - 0.5] = center + radius - 0.5

    return lms

def uniform_distribution_faces_cube(begin_corner, end_corner, n_points_dim):
    diff = np.array(end_corner) - np.array(begin_corner)
    lms = []

    for corner, mult in zip([begin_corner, end_corner], [1, -1]):
        for corner_displacement in [np.array(x) for x in [(1, 1, 0), (1, 0, 1), (0, 1, 1)]]:
            indices = list(np.nonzero(corner_displacement)[0])
            indices.append(np.where(corner_displacement == 0)[0][0])
            end_corner_ = corner + mult * corner_displacement * diff
            points = np.array([None, None, None])
            points[indices[0]] = np.linspace(corner[indices[0]], end_corner_[indices[0]], n_points_dim)
            points[indices[1]] = np.linspace(corner[indices[1]], end_corner_[indices[1]], n_points_dim)
            points[indices[2]] = [end_corner_[indices[2]]]
            
            lms.extend(list(itertools.product(*points)))

    return np.unique(np.array(lms), axis=0)

@np.vectorize
def cube_sphere_mapping(x, y, z):   
    return x*sqrt(1 - y**2/2 - z**2/2 + y**2*z**2/3), y*sqrt(1 - z**2/2 - x**2/2 + z**2*x**2/3), z*sqrt(1 - x**2/2 - y**2/2 + x**2*y**2/3)

def scale_and_translate(points, scale, translation):
    return points * scale + translation

def write_landmarks_to_file(*landmarks, filename):
    total_size = sum([len(l) for l in landmarks])
    with open(filename, 'w') as f:
        f.write('point\n')
        f.write(f'{total_size}\n')
        for landmarks_ in landmarks:
            for lm in landmarks_:
                f.write(f'{lm[0]} {lm[1]} {lm[2]}\n')                

In [105]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [145]:
lms = uniform_distribution_faces_cube((-1, -1, -1), (1, 1, 1), 7)
lms_plot = scale_and_translate(lms, 10, 14.5)
lms_plot[lms_plot < 5] = 5
lms_plot[lms_plot > 24] = 24
fig = px.scatter_3d(x=lms_plot[:,0], y=lms_plot[:,1], z=lms_plot[:,2])
fig.show()

In [146]:
lms_sphere = np.array(cube_sphere_mapping(*lms.T)).T
lms_sphere = scale_and_translate(lms_sphere, 7, 14.5)
lms_sphere[lms_sphere < 8] = 8
lms_sphere[lms_sphere > 21] = 21
fig = px.scatter_3d(x=lms_sphere[:,0], y=lms_sphere[:,1], z=lms_sphere[:,2])
fig.show()

In [147]:
unit_cube_points = uniform_distribution_faces_cube((-1, -1, -1), (1, 1, 1), 7)
unit_sphere_points = np.array(cube_sphere_mapping(*unit_cube_points.T)).T

# Scene 01

lms_cube = scale_and_translate(unit_cube_points, 10, 14.5)
lms_cube[lms_cube < 5] = 5
lms_cube[lms_cube > 24] = 24

lms_sphere_moving = scale_and_translate(unit_sphere_points, 7, 14.5)
lms_sphere_moving[lms_sphere_moving < 8] = 8
lms_sphere_moving[lms_sphere_moving > 21] = 21

lms_sphere_fixed = scale_and_translate(unit_sphere_points, 4, 14.5)
lms_sphere_fixed[lms_sphere_fixed < 11] = 11
lms_sphere_fixed[lms_sphere_fixed > 18] = 18

write_landmarks_to_file(lms_cube, lms_sphere_moving, filename='01_Moving.txt')
write_landmarks_to_file(lms_cube, lms_sphere_fixed, filename='01_Fixed.txt')

In [144]:
#  Scene 02

lms_cube_moving = scale_and_translate(unit_cube_points, 10, 14.5)
lms_cube_moving[lms_cube_moving < 5] = 5
lms_cube_moving[lms_cube_moving > 24] = 24

lms_cube_fixed = scale_and_translate(unit_cube_points, 6, 14.5)
lms_cube_fixed[lms_cube_fixed < 9] = 9
lms_cube_fixed[lms_cube_fixed > 20] = 20

lms_sphere_moving = scale_and_translate(unit_sphere_points, 5, 14.5)
lms_sphere_moving[lms_sphere_moving < 10] = 10
lms_sphere_moving[lms_sphere_moving > 19] = 19

lms_sphere_fixed = scale_and_translate(unit_sphere_points, 12, 14.5)
lms_sphere_fixed[lms_sphere_fixed < 3] = 3
lms_sphere_fixed[lms_sphere_fixed > 26] = 26

write_landmarks_to_file(lms_cube_moving, lms_sphere_moving, filename='02_Moving.txt')
write_landmarks_to_file(lms_sphere_fixed, lms_cube_fixed, filename='02_Fixed.txt')